<a href="https://colab.research.google.com/github/dayaiit/test/blob/main/L3_MML_C_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Systems of Linear Equations

This notebook implements the key algorithms and concepts for solving systems of linear equations, including:
- Matrix representation of linear systems
- Gaussian elimination
- Systems in echelon and row canonical form
- Homogeneous systems
- LU decomposition

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fractions import Fraction

# Set numpy to print full precision and suppress scientific notation
np.set_printoptions(precision=4, suppress=True)

# Function to format fractions nicely
def frac_str(f):
    if f.denominator == 1:
        return str(f.numerator)
    else:
        return f"{f.numerator}/{f.denominator}"

# Function to print a matrix with fractions
def print_matrix(matrix, name="Matrix"):
    print(f"\n{name}:")
    for row in matrix:
        print("[", end="")
        for i, val in enumerate(row):
            if i == len(row) - 1 and len(row) > matrix.shape[1]:  # Is this the augmented column?
                print(" |", end="")
            frac = Fraction(val).limit_denominator()
            print(f" {frac_str(frac)}", end="")
        print(" ]")

## 1. Matrix Representation of Linear Systems

A system of m linear equations in n unknowns can be represented as:

$$
\begin{align}
a_{11}x_1 + a_{12}x_2 + \ldots + a_{1n}x_n &= b_1 \\
a_{21}x_1 + a_{22}x_2 + \ldots + a_{2n}x_n &= b_2 \\
\vdots \\
a_{m1}x_1 + a_{m2}x_2 + \ldots + a_{mn}x_n &= b_m
\end{align}
$$

This can be written in matrix form as $AX = B$ where:
- $A$ is the coefficient matrix $[a_{ij}]$
- $X$ is the column vector of unknowns $[x_1, x_2, \ldots, x_n]^T$
- $B$ is the column vector of constants $[b_1, b_2, \ldots, b_m]^T$

The augmented matrix is $[A|B]$.

In [ ]:
class LinearSystem:
    """A class to represent and solve systems of linear equations"""

    def __init__(self, A, b=None):
        """Initialize with coefficient matrix A and constants b"""
        self.A = np.array(A, dtype=float)

        # If b is provided, store it, otherwise assume homogeneous system (b=0)
        if b is not None:
            self.b = np.array(b, dtype=float)
        else:
            self.b = np.zeros(self.A.shape[0])

        # Create augmented matrix [A|b]
        self.augmented = np.column_stack((self.A, self.b))

        # Store original matrices for reference
        self.original_A = self.A.copy()
        self.original_b = self.b.copy()
        self.original_augmented = self.augmented.copy()

    def __str__(self):
        """String representation of the system"""
        result = "System of Linear Equations:\n"
        n_vars = self.A.shape[1]

        for i, row in enumerate(self.A):
            terms = []
            for j, coef in enumerate(row):
                if coef == 0:
                    continue
                if coef == 1:
                    terms.append(f"x_{j+1}")
                elif coef == -1:
                    terms.append(f"-x_{j+1}")
                else:
                    frac = Fraction(coef).limit_denominator()
                    terms.append(f"{frac_str(frac)} x_{j+1}")

            equation = " + ".join(terms).replace("+ -", "- ")
            b_val = Fraction(self.b[i]).limit_denominator()
            result += f"{equation} = {frac_str(b_val)}\n"

        return result

    def display(self):
        """Display the system in matrix form"""
        print("\nCoefficient Matrix A:")
        print(self.A)
        print("\nConstants b:")
        print(self.b)
        print("\nAugmented Matrix [A|b]:")
        print(self.augmented)



In [ ]:
# Example: Create a system from a coefficient matrix and constants
A = np.array([
    [1, -3, -2],
    [2, -4, -3],
    [3, 6, 8]
])
b = np.array([6, 8, 5])

system = LinearSystem(A, b)
print(system)
system.display()

System of Linear Equations:
x_1 - 3 x_2 - 2 x_3 = 6
2 x_1 - 4 x_2 - 3 x_3 = 8
3 x_1 + 6 x_2 + 8 x_3 = 5


Coefficient Matrix A:
[[ 1. -3. -2.]
 [ 2. -4. -3.]
 [ 3.  6.  8.]]

Constants b:
[6. 8. 5.]

Augmented Matrix [A|b]:
[[ 1. -3. -2.  6.]
 [ 2. -4. -3.  8.]
 [ 3.  6.  8.  5.]]


## 2. Elementary Row Operations

There are three types of elementary row operations that preserve the solution set of a system:

1. **Interchange** two rows: $R_i \leftrightarrow R_j$
2. **Scale** a row by a nonzero constant: $kR_i \rightarrow R_i$ (where $k \neq 0$)
3. **Replace** a row by the sum of itself and a multiple of another row: $R_i + kR_j \rightarrow R_i$

Let's implement these operations:

In [ ]:
def add_row_operations_to_class(LinearSystem):
    def row_interchange(self, i, j):
        """Interchange rows i and j (0-indexed)"""
        if i != j:
            self.augmented[[i, j]] = self.augmented[[j, i]]
            self.A[[i, j]] = self.A[[j, i]]
            self.b[[i, j]] = self.b[[j, i]]
            print(f"R{i+1} ↔ R{j+1}")
        return self

    def row_scale(self, i, k):
        """Scale row i by k (0-indexed)"""
        if k != 0:
            self.augmented[i] *= k
            self.A[i] *= k
            self.b[i] *= k
            frac_k = Fraction(k).limit_denominator()
            print(f"{frac_str(frac_k)}R{i+1} → R{i+1}")
        return self

    def row_add(self, i, j, k):
        """Replace row i with row i + k*row j (0-indexed)"""
        if k != 0:
            self.augmented[i] += k * self.augmented[j]
            self.A[i] += k * self.A[j]
            self.b[i] += k * self.b[j]
            frac_k = Fraction(k).limit_denominator()
            if frac_k > 0:
                print(f"R{i+1} + {frac_str(frac_k)}R{j+1} → R{i+1}")
            else:
                print(f"R{i+1} - {frac_str(abs(frac_k))}R{j+1} → R{i+1}")
        return self

    LinearSystem.row_interchange = row_interchange
    LinearSystem.row_scale = row_scale
    LinearSystem.row_add = row_add
    return LinearSystem



In [ ]:
# Add the operations to our class
LinearSystem = add_row_operations_to_class(LinearSystem)

# Example: Apply elementary row operations
system = LinearSystem(A, b)
print_matrix(system.augmented, "Initial augmented matrix")

# Apply some row operations
system.row_add(1, 0, -2)  # R2 = R2 - 2*R1
print_matrix(system.augmented, "After R2 = R2 - 2*R1")

system.row_add(2, 0, -3)  # R3 = R3 - 3*R1
print_matrix(system.augmented, "After R3 = R3 - 3*R1")


Initial augmented matrix:
[ 1 -3 -2 6 ]
[ 2 -4 -3 8 ]
[ 3 6 8 5 ]
R2 - 2R1 → R2

After R2 = R2 - 2*R1:
[ 1 -3 -2 6 ]
[ 0 2 1 -4 ]
[ 3 6 8 5 ]
R3 - 3R1 → R3

After R3 = R3 - 3*R1:
[ 1 -3 -2 6 ]
[ 0 2 1 -4 ]
[ 0 15 14 -13 ]


## 3. Gaussian Elimination

Gaussian elimination converts a system into triangular (echelon) form through two main steps:

1. **Forward Elimination**: Convert the coefficient matrix to upper triangular form
2. **Back Substitution**: Solve for the unknowns starting from the last equation

Let's implement Gaussian elimination:

In [ ]:
def add_gaussian_elimination(LinearSystem):
    def forward_elimination(self):
        """Convert augmented matrix to row echelon form"""
        print("\n--- Forward Elimination ---")
        m, n = self.A.shape
        pivot_row = 0
        pivot_positions = []

        # For each column
        for j in range(n):
            # Skip if we've processed all rows
            if pivot_row >= m:
                break

            # Find the pivot (largest absolute value in the current column)
            pivot_idx = pivot_row + np.argmax(np.abs(self.augmented[pivot_row:, j]))

            # If the pivot is zero, move to the next column
            if np.isclose(self.augmented[pivot_idx, j], 0):
                continue

            # Swap rows if needed
            if pivot_idx != pivot_row:
                self.row_interchange(pivot_row, pivot_idx)

            # Record pivot position
            pivot_positions.append((pivot_row, j))

            # Eliminate below the pivot
            for i in range(pivot_row + 1, m):
                factor = -self.augmented[i, j] / self.augmented[pivot_row, j]
                if not np.isclose(factor, 0):
                    self.row_add(i, pivot_row, factor)

            # Move to the next row
            pivot_row += 1

        print("\nRow echelon form:")
        print_matrix(self.augmented)
        return pivot_positions

    def back_substitution(self, pivot_positions):
        """Solve the system given its row echelon form"""
        print("\n--- Back Substitution ---")
        n = self.A.shape[1]  # Number of variables
        m = len(pivot_positions)  # Number of non-zero rows (rank)

        # Check if the system is consistent
        for i in range(m, self.A.shape[0]):
            if not np.isclose(self.augmented[i, -1], 0):
                print("The system is inconsistent and has no solution.")
                return None

        # Initialize solution with free variables set to 0
        solution = np.zeros(n)
        free_vars = set(range(n)) - set(j for _, j in pivot_positions)

        # Work backwards from the last pivot
        for k in range(m-1, -1, -1):
            row, col = pivot_positions[k]

            # Calculate the value for this variable
            rhs = self.augmented[row, -1]  # Right-hand side value

            # Subtract the contribution of variables already solved
            for j in range(col + 1, n):
                rhs -= self.augmented[row, j] * solution[j]

            # Set the variable's value
            solution[col] = rhs / self.augmented[row, col]

        # Print the solution
        print("\nSolution:")
        if free_vars:
            print("The system has infinitely many solutions.")
            for j in range(n):
                if j in free_vars:
                    print(f"x_{j+1} = free variable")
                else:
                    frac = Fraction(solution[j]).limit_denominator()
                    print(f"x_{j+1} = {frac_str(frac)}")
        else:
            for j in range(n):
                frac = Fraction(solution[j]).limit_denominator()
                print(f"x_{j+1} = {frac_str(frac)}")

        return solution

    def solve_gaussian(self):
        """Solve the system using Gaussian elimination"""
        print("\nSolving using Gaussian elimination:")
        print_matrix(self.augmented, "Initial augmented matrix")

        # Forward elimination
        pivot_positions = self.forward_elimination()

        # Back substitution
        return self.back_substitution(pivot_positions)

    LinearSystem.forward_elimination = forward_elimination
    LinearSystem.back_substitution = back_substitution
    LinearSystem.solve_gaussian = solve_gaussian
    return LinearSystem

# Add the operations to our class
LinearSystem = add_gaussian_elimination(LinearSystem)

# Example: Solve a system using Gaussian elimination
system = LinearSystem(A, b)
solution = system.solve_gaussian()

# Verify the solution
if solution is not None:
    print("\nVerification:")
    residual = system.original_A @ solution - system.original_b
    print(f"Residual (should be close to zero): {residual}")


Solving using Gaussian elimination:

Initial augmented matrix:
[ 1 -3 -2 6 ]
[ 2 -4 -3 8 ]
[ 3 6 8 5 ]

--- Forward Elimination ---
R1 ↔ R3
R2 - 2/3R1 → R2
R3 - 1/3R1 → R3
R3 - 5/8R2 → R3

Row echelon form:

Matrix:
[ 3 6 8 5 ]
[ 0 -8 -25/3 14/3 ]
[ 0 0 13/24 17/12 ]

--- Back Substitution ---

Solution:
x_1 = 17/13
x_2 = -43/13
x_3 = 34/13

Verification:
Residual (should be close to zero): [ 0. -0.  0.]


## 4. Row Canonical Form (Gauss-Jordan Elimination)

Gauss-Jordan elimination extends Gaussian elimination to produce the reduced row echelon form (row canonical form):

1. Each leading nonzero entry (pivot) is 1
2. Each column containing a pivot has zeros in all other positions

Let's implement Gauss-Jordan elimination:

In [ ]:
def add_gauss_jordan(LinearSystem):
    def to_row_canonical_form(self):
        """Convert the system to row canonical form using Gauss-Jordan elimination"""
        print("\n--- Gauss-Jordan Elimination ---")
        print_matrix(self.augmented, "Initial augmented matrix")

        # Step 1: Get to row echelon form using forward elimination
        pivot_positions = self.forward_elimination()

        # Step 2: Scale each pivot row to make the pivot 1
        for row, col in pivot_positions:
            if not np.isclose(self.augmented[row, col], 1):
                self.row_scale(row, 1 / self.augmented[row, col])

        # Step 3: Zero out entries above each pivot
        for i, (row, col) in enumerate(pivot_positions):
            for r in range(row):
                if not np.isclose(self.augmented[r, col], 0):
                    self.row_add(r, row, -self.augmented[r, col])

        print("\nRow canonical form (reduced row echelon form):")
        print_matrix(self.augmented)

        return pivot_positions

    def solve_gauss_jordan(self):
        """Solve the system using Gauss-Jordan elimination"""
        pivot_positions = self.to_row_canonical_form()

        # Extract solution from row canonical form
        n = self.A.shape[1]  # Number of variables
        solution = np.zeros(n)

        # Check if the system is consistent
        for i in range(len(pivot_positions), self.A.shape[0]):
            if not np.isclose(self.augmented[i, -1], 0):
                print("\nThe system is inconsistent and has no solution.")
                return None

        # Identify pivot and free variables
        pivot_cols = [col for _, col in pivot_positions]
        free_vars = set(range(n)) - set(pivot_cols)

        # Extract solution directly from the canonical form
        for row, col in pivot_positions:
            solution[col] = self.augmented[row, -1]

        # Print the solution
        print("\nSolution:")
        if free_vars:
            print("The system has infinitely many solutions.")
            for j in range(n):
                if j in free_vars:
                    print(f"x_{j+1} = free variable")
                else:
                    frac = Fraction(solution[j]).limit_denominator()
                    print(f"x_{j+1} = {frac_str(frac)}")
        else:
            for j in range(n):
                frac = Fraction(solution[j]).limit_denominator()
                print(f"x_{j+1} = {frac_str(frac)}")

        return solution

    LinearSystem.to_row_canonical_form = to_row_canonical_form
    LinearSystem.solve_gauss_jordan = solve_gauss_jordan
    return LinearSystem

# Add the operations to our class
LinearSystem = add_gauss_jordan(LinearSystem)

# Example: Solve a system using Gauss-Jordan elimination
system = LinearSystem(A, b)
solution = system.solve_gauss_jordan()

# Verify the solution
if solution is not None:
    print("\nVerification:")
    residual = system.original_A @ solution - system.original_b
    print(f"Residual (should be close to zero): {residual}")


--- Gauss-Jordan Elimination ---

Initial augmented matrix:
[ 1 -3 -2 6 ]
[ 2 -4 -3 8 ]
[ 3 6 8 5 ]

--- Forward Elimination ---
R1 ↔ R3
R2 - 2/3R1 → R2
R3 - 1/3R1 → R3
R3 - 5/8R2 → R3

Row echelon form:

Matrix:
[ 3 6 8 5 ]
[ 0 -8 -25/3 14/3 ]
[ 0 0 13/24 17/12 ]
1/3R1 → R1
-1/8R2 → R2
24/13R3 → R3
R1 - 2R2 → R1
R1 - 7/12R3 → R1
R2 - 25/24R3 → R2

Row canonical form (reduced row echelon form):

Matrix:
[ 1 0 0 17/13 ]
[ 0 1 0 -43/13 ]
[ 0 0 1 34/13 ]

Solution:
x_1 = 17/13
x_2 = -43/13
x_3 = 34/13

Verification:
Residual (should be close to zero): [ 0. -0. -0.]


## 5. Homogeneous Systems

A homogeneous system has the form $AX = 0$. It always has the trivial solution $X = 0$, but may have nontrivial solutions if $\text{rank}(A) < n$.

In [ ]:
# Example: Homogeneous system
A_homog = np.array([
    [2, 4, -5, 3],
    [3, 6, -7, 4],
    [5, 10, -11, 6]
])

# Create and solve the homogeneous system
homogeneous_system = LinearSystem(A_homog)  # Default b is all zeros
print("Homogeneous system:")
print(homogeneous_system)
homogeneous_system.display()

print("\nSolving the homogeneous system:")
solution = homogeneous_system.solve_gauss_jordan()

# For homogeneous systems with free variables, let's show the general solution
if solution is not None:
    # Extract pivot and free variables
    pivot_positions = homogeneous_system.forward_elimination()
    pivot_cols = [col for _, col in pivot_positions]
    free_vars = list(set(range(A_homog.shape[1])) - set(pivot_cols))

    if free_vars:
        print("\nGeneral solution in terms of free variables:")
        # For each free variable, find the corresponding solution basis vector
        for free_var in free_vars:
            # Set this free variable to 1, others to 0
            temp_system = LinearSystem(A_homog)
            temp_system.to_row_canonical_form()

            basis_vector = np.zeros(A_homog.shape[1])
            basis_vector[free_var] = 1

            # Set the pivot variables to make the system consistent
            for row, col in pivot_positions:
                value = 0
                for j in free_vars:
                    value -= temp_system.augmented[row, j] * (1 if j == free_var else 0)
                basis_vector[col] = value

            # Print this basis vector
            print(f"\nBasis vector for free variable x_{free_var+1}:")
            for j in range(A_homog.shape[1]):
                frac = Fraction(basis_vector[j]).limit_denominator()
                print(f"x_{j+1} = {frac_str(frac)}")

            # Verify this is a solution
            residual = A_homog @ basis_vector
            print(f"Verification (residual should be near 0): {np.linalg.norm(residual)}")

Homogeneous system:
System of Linear Equations:
2 x_1 + 4 x_2 - 5 x_3 + 3 x_4 = 0
3 x_1 + 6 x_2 - 7 x_3 + 4 x_4 = 0
5 x_1 + 10 x_2 - 11 x_3 + 6 x_4 = 0


Coefficient Matrix A:
[[  2.   4.  -5.   3.]
 [  3.   6.  -7.   4.]
 [  5.  10. -11.   6.]]

Constants b:
[0. 0. 0.]

Augmented Matrix [A|b]:
[[  2.   4.  -5.   3.   0.]
 [  3.   6.  -7.   4.   0.]
 [  5.  10. -11.   6.   0.]]

Solving the homogeneous system:

--- Gauss-Jordan Elimination ---

Initial augmented matrix:
[ 2 4 -5 3 0 ]
[ 3 6 -7 4 0 ]
[ 5 10 -11 6 0 ]

--- Forward Elimination ---
R1 ↔ R3
R2 - 3/5R1 → R2
R3 - 2/5R1 → R3
R2 ↔ R3
R3 - 2/3R2 → R3

Row echelon form:

Matrix:
[ 5 10 -11 6 0 ]
[ 0 0 -3/5 3/5 0 ]
[ 0 0 0 0 0 ]
1/5R1 → R1
-5/3R2 → R2
R1 + 11/5R2 → R1

Row canonical form (reduced row echelon form):

Matrix:
[ 1 2 0 -1 0 ]
[ 0 0 1 -1 0 ]
[ 0 0 0 0 0 ]

Solution:
The system has infinitely many solutions.
x_1 = 0
x_2 = free variable
x_3 = 0
x_4 = free variable

--- Forward Elimination ---

Row echelon form:

Matrix:


## 6. LU Decomposition

LU decomposition factors a matrix A into the product of a lower triangular matrix L and an upper triangular matrix U: $A = LU$.

This is useful for solving multiple systems $AX = B$ with the same A but different B.

In [ ]:
def add_lu_decomposition(LinearSystem):
    def lu_decomposition(self):
        """Perform LU decomposition of the coefficient matrix A"""
        print("\n--- LU Decomposition ---")

        # Create copies to avoid modifying the original
        A = self.A.copy()
        n = A.shape[0]

        # Check if A is square
        if A.shape[0] != A.shape[1]:
            print("LU decomposition requires a square matrix.")
            return None, None

        # Initialize L as identity and U as zeros
        L = np.eye(n)
        U = np.zeros_like(A)

        # Perform elimination and collect multipliers in L
        for j in range(n):
            # U gets the current state of A for row j
            U[j, j:] = A[j, j:]

            # Skip if pivot is zero
            if abs(A[j, j]) < 1e-10:
                print(f"Pivot at ({j},{j}) is zero. LU decomposition without pivoting failed.")
                return None, None

            # Compute multipliers and update A
            for i in range(j+1, n):
                L[i, j] = A[i, j] / A[j, j]  # Store multiplier in L
                A[i, j:] -= L[i, j] * A[j, j:]  # Update A

        print("\nL matrix (lower triangular with 1's on diagonal):")
        print(L)

        print("\nU matrix (upper triangular):")
        print(U)

        # Verify decomposition
        print("\nVerification: L*U should equal original A")
        print("Original A:")
        print(self.original_A)
        print("L*U:")
        print(L @ U)
        print(f"Difference norm: {np.linalg.norm(self.original_A - L@U)}")

        return L, U

    def solve_using_lu(self, b):
        """Solve Ax = b given the LU decomposition of A"""
        # Get the LU decomposition
        L, U = self.lu_decomposition()
        if L is None or U is None:
            return None

        n = L.shape[0]
        b = np.array(b, dtype=float)

        # Step 1: Solve Ly = b for y using forward substitution
        y = np.zeros(n)
        for i in range(n):
            y[i] = b[i] - np.dot(L[i, :i], y[:i])

        # Step 2: Solve Ux = y for x using back substitution
        x = np.zeros(n)
        for i in range(n-1, -1, -1):
            x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]

        print("\nSolution:")
        for j in range(n):
            frac = Fraction(x[j]).limit_denominator()
            print(f"x_{j+1} = {frac_str(frac)}")

        return x

    LinearSystem.lu_decomposition = lu_decomposition
    LinearSystem.solve_using_lu = solve_using_lu
    return LinearSystem

# Add the operations to our class
LinearSystem = add_lu_decomposition(LinearSystem)

# Example: LU decomposition for a square system
A_square = np.array([
    [1, 2, 1],
    [2, 3, 3],
    [3, 10, 2]
])
b_square = np.array([3, 5, 17])

square_system = LinearSystem(A_square, b_square)
print("Square system:")
print(square_system)

# Solve using LU decomposition
solution = square_system.solve_using_lu(b_square)

# Verify the solution
if solution is not None:
    print("\nVerification:")
    residual = A_square @ solution - b_square
    print(f"Residual (should be close to zero): {residual}")

# Example: Solve the same system with multiple right-hand sides
b2 = np.array([2, 4, 16])
print("\nSolving with a different right-hand side:")
solution2 = square_system.solve_using_lu(b2)

Square system:
System of Linear Equations:
x_1 + 2 x_2 + x_3 = 3
2 x_1 + 3 x_2 + 3 x_3 = 5
3 x_1 + 10 x_2 + 2 x_3 = 17


--- LU Decomposition ---

L matrix (lower triangular with 1's on diagonal):
[[ 1.  0.  0.]
 [ 2.  1.  0.]
 [ 3. -4.  1.]]

U matrix (upper triangular):
[[ 1.  2.  1.]
 [ 0. -1.  1.]
 [ 0.  0.  3.]]

Verification: L*U should equal original A
Original A:
[[ 1.  2.  1.]
 [ 2.  3.  3.]
 [ 3. 10.  2.]]
L*U:
[[ 1.  2.  1.]
 [ 2.  3.  3.]
 [ 3. 10.  2.]]
Difference norm: 0.0

Solution:
x_1 = -3
x_2 = 7/3
x_3 = 4/3

Verification:
Residual (should be close to zero): [0. 0. 0.]

Solving with a different right-hand side:

--- LU Decomposition ---

L matrix (lower triangular with 1's on diagonal):
[[ 1.  0.  0.]
 [ 2.  1.  0.]
 [ 3. -4.  1.]]

U matrix (upper triangular):
[[ 1.  2.  1.]
 [ 0. -1.  1.]
 [ 0.  0.  3.]]

Verification: L*U should equal original A
Original A:
[[ 1.  2.  1.]
 [ 2.  3.  3.]
 [ 3. 10.  2.]]
L*U:
[[ 1.  2.  1.]
 [ 2.  3.  3.]
 [ 3. 10.  2.]]
Difference n

## 7. Practice Problems

Let's solve some practice problems using our implementation:

In [ ]:
# Practice Problem 1: A system with a unique solution
print("\n--- Practice Problem 1: System with unique solution ---")
A1 = np.array([
    [2, -5, 8],
    [3, 2, -1],
    [1, -4, 5]
])
b1 = np.array([21, 4, 12])

system1 = LinearSystem(A1, b1)
print(system1)

print("\nSolving with Gaussian elimination:")
solution1 = system1.solve_gaussian()

# Verify solution
if solution1 is not None:
    print("\nVerification:")
    residual1 = A1 @ solution1 - b1
    print(f"Residual: {residual1}")


--- Practice Problem 1: System with unique solution ---
System of Linear Equations:
2 x_1 - 5 x_2 + 8 x_3 = 21
3 x_1 + 2 x_2 - x_3 = 4
x_1 - 4 x_2 + 5 x_3 = 12


Solving with Gaussian elimination:

Solving using Gaussian elimination:

Initial augmented matrix:
[ 2 -5 8 21 ]
[ 3 2 -1 4 ]
[ 1 -4 5 12 ]

--- Forward Elimination ---
R1 ↔ R2
R2 - 2/3R1 → R2
R3 - 1/3R1 → R3
R3 - 14/19R2 → R3

Row echelon form:

Matrix:
[ 3 2 -1 4 ]
[ 0 -19/3 26/3 55/3 ]
[ 0 0 -20/19 -54/19 ]

--- Back Substitution ---

Solution:
x_1 = 17/10
x_2 = 4/5
x_3 = 27/10

Verification:
Residual: [0. 0. 0.]


In [ ]:
# Practice Problem 2: A system with no solution
print("\n--- Practice Problem 2: Inconsistent system (no solution) ---")
A2 = np.array([
    [1, -2, 3],
    [2, -4, 5],
    [3, -6, 8]
])
b2 = np.array([2, 3, 5])

system2 = LinearSystem(A2, b2)
print(system2)

print("\nSolving with Gauss-Jordan elimination:")
solution2 = system2.solve_gauss_jordan()


--- Practice Problem 2: Inconsistent system (no solution) ---
System of Linear Equations:
x_1 - 2 x_2 + 3 x_3 = 2
2 x_1 - 4 x_2 + 5 x_3 = 3
3 x_1 - 6 x_2 + 8 x_3 = 5


Solving with Gauss-Jordan elimination:

--- Gauss-Jordan Elimination ---

Initial augmented matrix:
[ 1 -2 3 2 ]
[ 2 -4 5 3 ]
[ 3 -6 8 5 ]

--- Forward Elimination ---
R1 ↔ R3
R2 - 2/3R1 → R2
R3 - 1/3R1 → R3
R2 ↔ R3
R3 + 1R2 → R3

Row echelon form:

Matrix:
[ 3 -6 8 5 ]
[ 0 0 1/3 1/3 ]
[ 0 0 0 0 ]
1/3R1 → R1
3R2 → R2
R1 - 8/3R2 → R1

Row canonical form (reduced row echelon form):

Matrix:
[ 1 -2 0 -1 ]
[ 0 0 1 1 ]
[ 0 0 0 0 ]

Solution:
The system has infinitely many solutions.
x_1 = -1
x_2 = free variable
x_3 = 1


In [ ]:
# Practice Problem 3: A system with infinitely many solutions
print("\n--- Practice Problem 3: System with infinitely many solutions ---")
A3 = np.array([
    [1, 2, 3, 4],
    [2, 4, 6, 8],
    [3, 6, 8, 10]
])
b3 = np.array([10, 20, 26])

system3 = LinearSystem(A3, b3)
print(system3)

print("\nSolving with Gaussian elimination:")
solution3 = system3.solve_gaussian()


--- Practice Problem 3: System with infinitely many solutions ---
System of Linear Equations:
x_1 + 2 x_2 + 3 x_3 + 4 x_4 = 10
2 x_1 + 4 x_2 + 6 x_3 + 8 x_4 = 20
3 x_1 + 6 x_2 + 8 x_3 + 10 x_4 = 26


Solving with Gaussian elimination:

Solving using Gaussian elimination:

Initial augmented matrix:
[ 1 2 3 4 10 ]
[ 2 4 6 8 20 ]
[ 3 6 8 10 26 ]

--- Forward Elimination ---
R1 ↔ R3
R2 - 2/3R1 → R2
R3 - 1/3R1 → R3
R3 - 1/2R2 → R3

Row echelon form:

Matrix:
[ 3 6 8 10 26 ]
[ 0 0 2/3 4/3 8/3 ]
[ 0 0 0 0 0 ]

--- Back Substitution ---

Solution:
The system has infinitely many solutions.
x_1 = -2
x_2 = free variable
x_3 = 4
x_4 = free variable


In [ ]:
# Practice Problem 4: Homogeneous system
print("\n--- Practice Problem 4: Homogeneous system ---")
A4 = np.array([
    [1, 2, -3, 4],
    [2, 4, -6, 8],
    [3, 6, -9, 12]
])

system4 = LinearSystem(A4)  # Homogeneous system (b = 0)
print(system4)

print("\nSolving with Gauss-Jordan elimination:")
solution4 = system4.solve_gauss_jordan()


--- Practice Problem 4: Homogeneous system ---
System of Linear Equations:
x_1 + 2 x_2 - 3 x_3 + 4 x_4 = 0
2 x_1 + 4 x_2 - 6 x_3 + 8 x_4 = 0
3 x_1 + 6 x_2 - 9 x_3 + 12 x_4 = 0


Solving with Gauss-Jordan elimination:

--- Gauss-Jordan Elimination ---

Initial augmented matrix:
[ 1 2 -3 4 0 ]
[ 2 4 -6 8 0 ]
[ 3 6 -9 12 0 ]

--- Forward Elimination ---
R1 ↔ R3
R2 - 2/3R1 → R2
R3 - 1/3R1 → R3

Row echelon form:

Matrix:
[ 3 6 -9 12 0 ]
[ 0 0 0 0 0 ]
[ 0 0 0 0 0 ]
1/3R1 → R1

Row canonical form (reduced row echelon form):

Matrix:
[ 1 2 -3 4 0 ]
[ 0 0 0 0 0 ]
[ 0 0 0 0 0 ]

Solution:
The system has infinitely many solutions.
x_1 = 0
x_2 = free variable
x_3 = free variable
x_4 = free variable


In [ ]:
# Practice Problem 5: Solving a system with LU decomposition
print("\n--- Practice Problem 5: LU decomposition ---")
A5 = np.array([
    [4, 3, 2],
    [2, 1, 3],
    [3, 4, 1]
])
b5 = np.array([10, 7, 8])

system5 = LinearSystem(A5, b5)
print(system5)

print("\nSolving with LU decomposition:")
solution5 = system5.solve_using_lu(b5)

# Verify solution
if solution5 is not None:
    print("\nVerification:")
    residual5 = A5 @ solution5 - b5
    print(f"Residual: {residual5}")


--- Practice Problem 5: LU decomposition ---
System of Linear Equations:
4 x_1 + 3 x_2 + 2 x_3 = 10
2 x_1 + x_2 + 3 x_3 = 7
3 x_1 + 4 x_2 + x_3 = 8


Solving with LU decomposition:

--- LU Decomposition ---

L matrix (lower triangular with 1's on diagonal):
[[ 1.    0.    0.  ]
 [ 0.5   1.    0.  ]
 [ 0.75 -3.5   1.  ]]

U matrix (upper triangular):
[[ 4.   3.   2. ]
 [ 0.  -0.5  2. ]
 [ 0.   0.   6.5]]

Verification: L*U should equal original A
Original A:
[[4. 3. 2.]
 [2. 1. 3.]
 [3. 4. 1.]]
L*U:
[[4. 3. 2.]
 [2. 1. 3.]
 [3. 4. 1.]]
Difference norm: 0.0

Solution:
x_1 = 19/13
x_2 = 8/13
x_3 = 15/13

Verification:
Residual: [0. 0. 0.]


## 8. Additional Exercises for Students

Here are some additional exercises you can try:

1. Modify the example from Gaussian elimination to use partial pivoting (already implemented)
2. Implement a function to determine the rank of a matrix
3. Develop a function to find a basis for the null space of a matrix
4. Create a visualization for the solution of a 2×2 or 3×3 system
5. Implement iterative methods like Jacobi or Gauss-Seidel for solving large systems
6. Compare the efficiency of different methods for solving systems

Example code for exercise #2 (finding the rank of a matrix):

In [ ]:
def matrix_rank(A):
    """Compute the rank of matrix A using Gaussian elimination"""
    # Create a system and reduce to row echelon form
    system = LinearSystem(A)
    pivot_positions = system.forward_elimination()

    # Rank is the number of pivot positions
    rank = len(pivot_positions)

    print(f"The rank of the matrix is: {rank}")
    return rank

# Test the rank function
print("\n--- Rank of a Matrix ---")
A_full_rank = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
A_rank_deficient = np.array([
    [1, 2, 3],
    [2, 4, 6],
    [3, 6, 9]
])

print("\nMatrix A_full_rank:")
print(A_full_rank)
rank1 = matrix_rank(A_full_rank)
print(f"NumPy computed rank: {np.linalg.matrix_rank(A_full_rank)}")

print("\nMatrix A_rank_deficient:")
print(A_rank_deficient)
rank2 = matrix_rank(A_rank_deficient)
print(f"NumPy computed rank: {np.linalg.matrix_rank(A_rank_deficient)}")


--- Rank of a Matrix ---

Matrix A_full_rank:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

--- Forward Elimination ---
R1 ↔ R3
R2 - 4/7R1 → R2
R3 - 1/7R1 → R3
R2 ↔ R3
R3 - 1/2R2 → R3

Row echelon form:

Matrix:
[ 7 8 9 0 ]
[ 0 6/7 12/7 0 ]
[ 0 0 0 0 ]
The rank of the matrix is: 2
NumPy computed rank: 2

Matrix A_rank_deficient:
[[1 2 3]
 [2 4 6]
 [3 6 9]]

--- Forward Elimination ---
R1 ↔ R3
R2 - 2/3R1 → R2
R3 - 1/3R1 → R3

Row echelon form:

Matrix:
[ 3 6 9 0 ]
[ 0 0 0 0 ]
[ 0 0 0 0 ]
The rank of the matrix is: 1
NumPy computed rank: 1


## Conclusion

In this notebook, we've implemented and demonstrated:

1. Basic representation of linear systems
2. Elementary row operations
3. Gaussian elimination
4. Gauss-Jordan elimination (row canonical form)
5. Solutions of homogeneous systems
6. LU decomposition
7. Practical examples and exercises

These tools form the foundation for solving systems of linear equations and are fundamental to many applications in science, engineering, and computer science.